# ResNet20 and Static Quantization of ResNet20 with CIRAF-10 DataSet

###  This notebook consist of: 
#### 1 task   -  create ResNet20 with Ciraf-10 and gain 90%> accuracy
#### 2 task   -  create quantized ResNet20 
#### 3 task   -  compare quantized and non quantized ResNet20 
#### 4 task   -  write own PTQ 

## TASK 1
###### create ResNet20 with Ciraf-10 and gain 90%> accuracy

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import time
import os
from torch.quantization import QuantStub
from torch.quantization import DeQuantStub

###  Device configuration with hyper-parametrs

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 200
learning_rate = 0.001

###  Load CIRAF10 dataset  with some transform

In [3]:

# Image preprocessing modules
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    normalize])
    
test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize])

train_dataset = torchvision.datasets.CIFAR10(root='data/',train=True, transform=train_transform,download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/',train=False, transform=test_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=100, shuffle=False)


Files already downloaded and verified


## Write basic Architecture for ResNet

In [4]:

def conv3x3(in_channels, out_channels, stride=1):
    """
    return 3x3 Conv2d
    """
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)


class ResidualBlock(nn.Module):
    """
    Initialize basic ResidualBlock with forward propogation
    """
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    """
    Initialize  ResNet with forward propogation
    """
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


###  Initialize ResNet20

In [5]:
model = ResNet(ResidualBlock, [3, 3, 3]).to(device)
model.eval();

###  Initialize optimizer and Loss function

In [6]:

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,)


In [7]:
# For updating learning rate
def update_lr(optimizer, lr):
    """
    This method update learning rate
    """
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [8]:
def evaluate(model):
    """
    Evaluate accuracy of test set and save weight of model
    """
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model_weight/'+str(int(100 * correct / total))+'resnet.ckpt')

### here we train our model with 200 epochs and every 30 epoch we divide learning rate by 3

In [9]:
# Train the model
total_step = len(train_loader)
curr_lr = 0.01
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    if((epoch+1)%5==0):
            evaluate(model)
    
    # Decay learning rate
    if (epoch+1) % 30 == 0:
        curr_lr /= 4
        update_lr(optimizer, curr_lr)


Epoch [1/200], Step [100/500] Loss: 1.7471
Epoch [1/200], Step [200/500] Loss: 1.4645
Epoch [1/200], Step [300/500] Loss: 1.3304
Epoch [1/200], Step [400/500] Loss: 1.1827
Epoch [1/200], Step [500/500] Loss: 1.3481
Epoch [2/200], Step [100/500] Loss: 1.0079
Epoch [2/200], Step [200/500] Loss: 1.1003
Epoch [2/200], Step [300/500] Loss: 1.2411
Epoch [2/200], Step [400/500] Loss: 0.9906
Epoch [2/200], Step [500/500] Loss: 0.9111
Epoch [3/200], Step [100/500] Loss: 0.9261
Epoch [3/200], Step [200/500] Loss: 0.8142
Epoch [3/200], Step [300/500] Loss: 1.0055
Epoch [3/200], Step [400/500] Loss: 0.6184
Epoch [3/200], Step [500/500] Loss: 0.7878
Epoch [4/200], Step [100/500] Loss: 0.8733
Epoch [4/200], Step [200/500] Loss: 0.6974
Epoch [4/200], Step [300/500] Loss: 0.7734
Epoch [4/200], Step [400/500] Loss: 0.8684
Epoch [4/200], Step [500/500] Loss: 0.6766
Epoch [5/200], Step [100/500] Loss: 0.6980
Epoch [5/200], Step [200/500] Loss: 0.7763
Epoch [5/200], Step [300/500] Loss: 0.7051
Epoch [5/20

Epoch [37/200], Step [100/500] Loss: 0.3856
Epoch [37/200], Step [200/500] Loss: 0.7130
Epoch [37/200], Step [300/500] Loss: 0.4029
Epoch [37/200], Step [400/500] Loss: 0.3563
Epoch [37/200], Step [500/500] Loss: 0.5844
Epoch [38/200], Step [100/500] Loss: 0.5668
Epoch [38/200], Step [200/500] Loss: 0.4222
Epoch [38/200], Step [300/500] Loss: 0.3127
Epoch [38/200], Step [400/500] Loss: 0.7031
Epoch [38/200], Step [500/500] Loss: 0.3118
Epoch [39/200], Step [100/500] Loss: 0.3097
Epoch [39/200], Step [200/500] Loss: 0.4674
Epoch [39/200], Step [300/500] Loss: 0.3497
Epoch [39/200], Step [400/500] Loss: 0.6048
Epoch [39/200], Step [500/500] Loss: 0.6647
Epoch [40/200], Step [100/500] Loss: 0.3816
Epoch [40/200], Step [200/500] Loss: 0.5220
Epoch [40/200], Step [300/500] Loss: 0.4080
Epoch [40/200], Step [400/500] Loss: 0.2890
Epoch [40/200], Step [500/500] Loss: 0.4997
Accuracy of the model on the test images: 83.16 %
Epoch [41/200], Step [100/500] Loss: 0.5375
Epoch [41/200], Step [200/

Epoch [72/200], Step [500/500] Loss: 0.1043
Epoch [73/200], Step [100/500] Loss: 0.1283
Epoch [73/200], Step [200/500] Loss: 0.1639
Epoch [73/200], Step [300/500] Loss: 0.2541
Epoch [73/200], Step [400/500] Loss: 0.1218
Epoch [73/200], Step [500/500] Loss: 0.2491
Epoch [74/200], Step [100/500] Loss: 0.1599
Epoch [74/200], Step [200/500] Loss: 0.1420
Epoch [74/200], Step [300/500] Loss: 0.1172
Epoch [74/200], Step [400/500] Loss: 0.1831
Epoch [74/200], Step [500/500] Loss: 0.1394
Epoch [75/200], Step [100/500] Loss: 0.0612
Epoch [75/200], Step [200/500] Loss: 0.1522
Epoch [75/200], Step [300/500] Loss: 0.2052
Epoch [75/200], Step [400/500] Loss: 0.2434
Epoch [75/200], Step [500/500] Loss: 0.1215
Accuracy of the model on the test images: 88.56 %
Epoch [76/200], Step [100/500] Loss: 0.1265
Epoch [76/200], Step [200/500] Loss: 0.1462
Epoch [76/200], Step [300/500] Loss: 0.2420
Epoch [76/200], Step [400/500] Loss: 0.2146
Epoch [76/200], Step [500/500] Loss: 0.0961
Epoch [77/200], Step [100/

Epoch [108/200], Step [300/500] Loss: 0.1703
Epoch [108/200], Step [400/500] Loss: 0.1660
Epoch [108/200], Step [500/500] Loss: 0.1534
Epoch [109/200], Step [100/500] Loss: 0.1096
Epoch [109/200], Step [200/500] Loss: 0.1756
Epoch [109/200], Step [300/500] Loss: 0.0795
Epoch [109/200], Step [400/500] Loss: 0.1699
Epoch [109/200], Step [500/500] Loss: 0.1786
Epoch [110/200], Step [100/500] Loss: 0.1191
Epoch [110/200], Step [200/500] Loss: 0.0910
Epoch [110/200], Step [300/500] Loss: 0.1062
Epoch [110/200], Step [400/500] Loss: 0.0926
Epoch [110/200], Step [500/500] Loss: 0.0752
Accuracy of the model on the test images: 89.64 %
Epoch [111/200], Step [100/500] Loss: 0.1139
Epoch [111/200], Step [200/500] Loss: 0.1479
Epoch [111/200], Step [300/500] Loss: 0.0510
Epoch [111/200], Step [400/500] Loss: 0.0682
Epoch [111/200], Step [500/500] Loss: 0.1135
Epoch [112/200], Step [100/500] Loss: 0.1319
Epoch [112/200], Step [200/500] Loss: 0.1824
Epoch [112/200], Step [300/500] Loss: 0.1641
Epoch

Epoch [143/200], Step [300/500] Loss: 0.0911
Epoch [143/200], Step [400/500] Loss: 0.1690
Epoch [143/200], Step [500/500] Loss: 0.0457
Epoch [144/200], Step [100/500] Loss: 0.0361
Epoch [144/200], Step [200/500] Loss: 0.0648
Epoch [144/200], Step [300/500] Loss: 0.0900
Epoch [144/200], Step [400/500] Loss: 0.0458
Epoch [144/200], Step [500/500] Loss: 0.1025
Epoch [145/200], Step [100/500] Loss: 0.1129
Epoch [145/200], Step [200/500] Loss: 0.0759
Epoch [145/200], Step [300/500] Loss: 0.0957
Epoch [145/200], Step [400/500] Loss: 0.1101
Epoch [145/200], Step [500/500] Loss: 0.0704
Accuracy of the model on the test images: 89.62 %
Epoch [146/200], Step [100/500] Loss: 0.1502
Epoch [146/200], Step [200/500] Loss: 0.0215
Epoch [146/200], Step [300/500] Loss: 0.0689
Epoch [146/200], Step [400/500] Loss: 0.0763
Epoch [146/200], Step [500/500] Loss: 0.0787
Epoch [147/200], Step [100/500] Loss: 0.0885
Epoch [147/200], Step [200/500] Loss: 0.1238
Epoch [147/200], Step [300/500] Loss: 0.0630
Epoch

Epoch [178/200], Step [300/500] Loss: 0.1364
Epoch [178/200], Step [400/500] Loss: 0.0894
Epoch [178/200], Step [500/500] Loss: 0.1601
Epoch [179/200], Step [100/500] Loss: 0.1559
Epoch [179/200], Step [200/500] Loss: 0.1063
Epoch [179/200], Step [300/500] Loss: 0.0913
Epoch [179/200], Step [400/500] Loss: 0.1244
Epoch [179/200], Step [500/500] Loss: 0.0519
Epoch [180/200], Step [100/500] Loss: 0.0624
Epoch [180/200], Step [200/500] Loss: 0.1002
Epoch [180/200], Step [300/500] Loss: 0.1388
Epoch [180/200], Step [400/500] Loss: 0.0772
Epoch [180/200], Step [500/500] Loss: 0.1365
Accuracy of the model on the test images: 89.67 %
Epoch [181/200], Step [100/500] Loss: 0.0665
Epoch [181/200], Step [200/500] Loss: 0.1398
Epoch [181/200], Step [300/500] Loss: 0.0691
Epoch [181/200], Step [400/500] Loss: 0.2209
Epoch [181/200], Step [500/500] Loss: 0.1123
Epoch [182/200], Step [100/500] Loss: 0.1190
Epoch [182/200], Step [200/500] Loss: 0.0476
Epoch [182/200], Step [300/500] Loss: 0.0523
Epoch

### in total we got the highest accuracy with 89.64 %

# TASK 2 
####  create quantized ResNet20 


### Write  Architecture for Quantized ResNet

In [101]:

def conv3x3(in_channels, out_channels, stride=1):
    """
    return 3x3 Conv2d
    """
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)


class QuantizedResidualBlock(nn.Module):
    """
    Initialize basic QuantizedResidualBlock with forward propogation
    """
    def __init__(self, in_channels, out_channels, stride=1, downsample=None,quantized=True):
        super(QuantizedResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.quantized = quantized
        self.skip_add = nn.quantized.FloatFunctional()
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        if self.quantized:
            out = self.skip_add.add(out, residual)
        else:
            out += residual  
        out = self.relu(out)
        return out


class QuantizedResNet(nn.Module):
    """
    Initialize  ResNet with forward propogation
    """
    def __init__(self, block, layers, num_classes=10):
        super(QuantizedResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.quant(x)
        out = self.conv(out)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.dequant(out)
        return out


### Initialize ResNet20  with highest accuracy

In [102]:
model = ResNet(ResidualBlock, [3, 3, 3]).to(device)
model.load_state_dict(torch.load('model_weight/89resnet.ckpt'))
model.eval();


### Initialize Quantized ResNet20  with highest accuracy

In [103]:
model_quantized =QuantizedResNet(QuantizedResidualBlock, [3, 3, 3]).to('cpu')
model_quantized.load_state_dict(torch.load('model_weight/89resnet.ckpt'))
model_quantized.eval();

In [104]:


#fuse layers
modules_to_fuse = [['conv', 'bn'],
                   ['layer1.0.conv1', 'layer1.0.bn1'],
                   ['layer1.0.conv2', 'layer1.0.bn2'],
                   ['layer1.1.conv1', 'layer1.1.bn1'],
                   ['layer1.1.conv2', 'layer1.1.bn2'],
                   ['layer1.2.conv1', 'layer1.2.bn1'],
                   ['layer1.2.conv2', 'layer1.2.bn2'],
                   
                   ['layer2.0.conv1', 'layer2.0.bn1'],
                   ['layer2.0.conv2', 'layer2.0.bn2'],
                   ['layer2.0.downsample.0', 'layer2.0.downsample.1'],
                   ['layer2.1.conv1', 'layer2.1.bn1'],
                   ['layer2.1.conv2', 'layer2.1.bn2'],
                   ['layer2.2.conv1', 'layer2.2.bn1'],
                   ['layer2.2.conv2', 'layer2.2.bn2'],
                   
                   ['layer3.0.conv1', 'layer3.0.bn1'],
                   ['layer3.0.conv2', 'layer3.0.bn2'],
                   ['layer3.0.downsample.0', 'layer3.0.downsample.1'],
                   ['layer3.1.conv1', 'layer3.1.bn1'],
                   ['layer3.1.conv2', 'layer3.1.bn2'],
                   ['layer3.2.conv1', 'layer3.2.bn1'],
                   ['layer3.2.conv2', 'layer3.2.bn2']]

model_quantized = torch.quantization.fuse_modules(model_quantized, modules_to_fuse)
model_quantized.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model_quantized, inplace=True)
model_quantized.eval();

for i, (images, labels) in enumerate(train_loader):
    images = images.to('cpu')
    model_quantized(images)
torch.quantization.convert(model_quantized, inplace=True)

# TASK 3
#### compare quantized and non quantized ResNet20 

# Script for total result

In [109]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print('Size (KB):', size/1e3)
    os.remove('temp.p')
    
def evaluate(model,device):
    """
    Evaluate accuracy of test set and save weight of model
    """
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return (100 * correct / total)


def test(model, device, test_loader):
    model.to(device)
    model.eval();
    print_size_of_model(model)
    st = time.time()
    accuracy=evaluate(model,device)
    et = time.time()
    ts=int((et - st) * 1000)
    print(ts)
    print("========================================= PERFORMANCE =============================================")
    print('Accuracy:  ({:.2f}%)\n'.format(accuracy))
    print('Elapsed time = {:0.4f} milliseconds'.format(ts))
    print("====================================================================================================")
    

# RESNET20

In [113]:
device = 'cpu'
test(model, device, test_loader) 

Size (KB): 1220.719
20247
========================================= PERFORMANCE =============================================
Accuracy:  (89.64%)

Elapsed time = 20247.0000 milliseconds


# Quantized RESNET20

In [112]:
device = 'cpu'
test(model_quantized, device, test_loader)

Size (KB): 363.007
10351
========================================= PERFORMANCE =============================================
Accuracy:  (87.98%)

Elapsed time = 10351.0000 milliseconds


#  task №4 не успел:)